In [1]:
import json
import urllib.request
import urllib.parse
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests

In [ ]:
def none_data(urdata, user_info) :
    user_status = pd.DataFrame({
                    'urname' : [urname], 
                    'adventurename' : None,
                    'adventurelevel' : None,
                    'resist' : None,
                    'physical_defense' : None,
                    'magical_defense' : None
                    })
    user_info = pd.concat([user_info, user_status])

In [74]:
urdata = pd.read_csv('2.1_final_result.csv', encoding = 'euc-kr')
urdata = urdata.dropna()

user_info = pd.DataFrame(columns = ['urname','adventurename', 'adventurelevel', 'resist', 'physical_defense', 'magical_defense'])
user_status = pd.DataFrame(columns = ['urname','adventurename', 'adventurelevel', 'resist', 'physical_defense', 'magical_defense'])

for i in range (167171, len(urdata)) :
    urname = urdata.iloc[i,0]
    svname = urdata.iloc[i,1]
    chid = urdata.iloc[i,3]
    processpoint = (i+1)/len(urdata)*100
    requestData = requests.get(f'https://api.neople.co.kr/df/servers/{svname}/characters/{chid}/status?apikey=M23RMaIB2JlKPyRWYRBKDb6UtE3NDTrS')
    if requestData.status_code == 200 :
        jsonData = requestData.json()
        api_result = pd.json_normalize(jsonData)
        if api_result.empty :
            user_status = pd.DataFrame({
                    'urname' : [urname], 
                    'adventurename' : None,
                    'adventurelevel' : None,
                    'resist' : None,
                    'physical_defense' : None,
                    'magical_defense' : None
                    })
            user_info = pd.concat([user_info, user_status])
        elif api_result.loc[0, 'adventureName'] == None :
            user_status = pd.DataFrame({
                    'urname' : [urname], 
                    'adventurename' : None,
                    'adventurelevel' : None,
                    'resist' : None,
                    'physical_defense' : None,
                    'magical_defense' : None
                    })
            user_info = pd.concat([user_info, user_status])
        elif api_result.loc[0, 'status'] == [] :
            user_status = pd.DataFrame({
                    'urname' : [urname], 
                    'adventurename' : None,
                    'adventurelevel' : None,
                    'resist' : None,
                    'physical_defense' : None,
                    'magical_defense' : None
                    })
            user_info = pd.concat([user_info, user_status])
        elif api_result.loc[0, 'level'] !=  110 :
            user_status = pd.DataFrame({
                    'urname' : [urname], 
                    'adventurename' : None,
                    'adventurelevel' : None,
                    'resist' : None,
                    'physical_defense' : None,
                    'magical_defense' : None
                    })
            user_info = pd.concat([user_info, user_status])

        else :
            # 모험단명 추출
            adventurename = api_result.loc[0,'adventureName']
            # Status 항목 추출
            status = pd.json_normalize(jsonData, record_path = 'status')
            status = status.set_index('name')
            total_resist = status.loc['화속성 저항', 'value'] + status.loc['수속성 저항', 'value'] + status.loc['명속성 저항', 'value'] + status.loc['암속성 저항', 'value']
            physical_def = status.loc['물리 방어율', 'value']
            magical_def = status.loc['마법 방어율', 'value']   
            #Buff 항목 추출
            buff = pd.json_normalize(jsonData, record_path = 'buff')
            buff = buff.set_index('name')
            adventureLevel = buff.loc['모험단 버프' , 'level']
            user_status = pd.DataFrame({'urname' : [urname], 
                    'adventurename' : [adventurename],
                    'adventurelevel' : [adventureLevel],
                    'resist' : [total_resist],
                    'physical_defense' : [physical_def],
                    'magical_defense' : [magical_def]})
            user_info = pd.concat([user_info, user_status])
            print("%0.2f%%, %s번째 확인중입니다"%(processpoint, i+1))
    else :
        none_data(urname,user_info)
        user_info = pd.concat([user_info, user_status])
        print("%0.2f%%, %s번째 확인중입니다"%(processpoint, i+1))
total_user_status = pd.merge(urdata, user_info, on = 'urname')
total_user_status.to_csv('2.2_Status_result.csv' , encoding = 'euc-kr')
print('완료했습니다.')

77.21%, 167172번째 확인중입니다
77.21%, 167173번째 확인중입니다
77.21%, 167174번째 확인중입니다
77.21%, 167175번째 확인중입니다
77.21%, 167176번째 확인중입니다
77.21%, 167177번째 확인중입니다
77.21%, 167178번째 확인중입니다
77.21%, 167180번째 확인중입니다
77.21%, 167181번째 확인중입니다
77.21%, 167182번째 확인중입니다
77.21%, 167183번째 확인중입니다
77.21%, 167184번째 확인중입니다
77.21%, 167185번째 확인중입니다
77.22%, 167186번째 확인중입니다
77.22%, 167187번째 확인중입니다
77.22%, 167188번째 확인중입니다
77.22%, 167189번째 확인중입니다
77.22%, 167190번째 확인중입니다
77.22%, 167191번째 확인중입니다
77.22%, 167192번째 확인중입니다
77.22%, 167193번째 확인중입니다
77.22%, 167194번째 확인중입니다
77.22%, 167195번째 확인중입니다
77.22%, 167196번째 확인중입니다
77.22%, 167197번째 확인중입니다
77.22%, 167198번째 확인중입니다
77.22%, 167199번째 확인중입니다
77.22%, 167200번째 확인중입니다
77.22%, 167201번째 확인중입니다
77.22%, 167202번째 확인중입니다
77.22%, 167203번째 확인중입니다
77.22%, 167204번째 확인중입니다
77.22%, 167205번째 확인중입니다
77.22%, 167206번째 확인중입니다
77.22%, 167207번째 확인중입니다
77.23%, 167208번째 확인중입니다
77.23%, 167209번째 확인중입니다
77.23%, 167210번째 확인중입니다
77.23%, 167211번째 확인중입니다
77.23%, 167212번째 확인중입니다
77.23%, 167213번째 확인중입니다
77.23%, 167214번째

In [72]:
api_result

,characterId,characterName,level,jobId,jobGrowId,jobName,jobGrowName,adventureName,guildId,guildName,buff,status
0,7f8ffc8a9076edc81b996397183a0f9c,레이저바죽하,110,afdf3b989339de478e85b614d274d1ef,92da05ec93fb43406e193ffb9a2a629b,거너(남),眞 어썰트,에픽풀세트,19523a6d5ad022e3d8245237a491d461,디어사이드™,"[{'name': '모험단 버프', 'level': 39, 'status': [{'...","[{'name': 'HP', 'value': 118254}, {'name': 'MP..."


In [32]:
if api_result.loc[0, 'level'] == '110':
# 모험단명 추출
    adventurename = api_result.loc[0,'adventureName']
    print(adventurename)
else :
    print(api_result.loc[0, 'level'])

84


In [87]:
user_info = pd.read_csv('Status_result.csv', encoding = 'euc-kr')
user_info = user_info.dropna()
total_user_status = pd.merge(urdata, user_info, on = 'urname')
total_user_status = total_user_status.drop_duplicates(subset='urname')
total_user_status = total_user_status.set_index('urname')
total_user_status.to_csv('2.2_Status_result.csv' , encoding = 'euc-kr')

In [89]:
total_user_status

,svname,rkpo,characterId,adventurename,adventurelevel,resist,physical_defense,magical_defense
urname,,,,,,,,
ID결신,bakal,"41,002",4663274c8e493bb43274ce15768a8cb9,축복의버프,40.0,79.0,37.6,40.0
연단된총,prey,"41,001",45737067e047d0ee8dc255c53f150a68,심연부터광명까지,37.0,54.0,38.1,42.0
킴던던,cain,"41,001",4d0d7052aa62f8899489c9ad0011267c,던던세상,38.0,89.0,41.9,45.1
구슬모양,cain,"41,001",4d9cccd884cbb336d9c9f3fe48a13457,싸인해주는토끼,39.0,74.0,37.0,34.2
딜좀이놈들아,prey,"41,001",3f979163c3355c45e6e8e4dd5040a52e,이게ㄹㅇ게임이냐,40.0,99.0,40.3,37.1
...,...,...,...,...,...,...,...,...
스핏라챠,cain,"33,038",e477e2caa27c0e8b7071fcfad04e6f35,멀개,37.0,69.0,28.9,33.8
Puming,bakal,"33,038",f68ac5ee580b409322070dea0af32a87,퓨밍,37.0,159.0,28.7,33.5
안보이천,anton,"33,038",5e2a846591fc3f290779ea19f5b4bdd8,알테리온,39.0,161.0,33.0,30.9


In [84]:
user_info.drop_duplicates(subset='urname')

,urname,adventurename,adventurelevel,resist,physical_defense,magical_defense
0,ID결신,축복의버프,40.0,79.0,37.6,40.0
1,연단된총,심연부터광명까지,37.0,54.0,38.1,42.0
2,킴던던,던던세상,38.0,89.0,41.9,45.1
3,구슬모양,싸인해주는토끼,39.0,74.0,37.0,34.2
4,딜좀이놈들아,이게ㄹㅇ게임이냐,40.0,99.0,40.3,37.1
...,...,...,...,...,...,...
216515,스핏라챠,멀개,37.0,69.0,28.9,33.8
216516,Puming,퓨밍,37.0,159.0,28.7,33.5
216517,안보이천,알테리온,39.0,161.0,33.0,30.9
216518,도짬,천장군,38.0,69.0,32.9,31.1
